In [ ]:
! pip install PyPDF2
! pip install chromadb
! pip install tavily
! pip install nemoguardrails
! pip install langchain openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 25.3 MB/s e

In [ ]:
from openai import OpenAI
import gradio as gr
import os
import PyPDF2
import chromadb
from dotenv import load_dotenv
load_dotenv()
import requests
from urllib.parse import urlparse
import re
import json
from datetime import datetime
import uuid
from sentence_transformers import SentenceTransformer, util
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [ ]:
os.environ["OPENAI_API_KEY"] = "" #Add your OPENAI API Key
load_dotenv()
from openai import OpenAI
client = OpenAI(api_key="")

In [ ]:
# LangSmith imports with error handling
os.environ["LANGCHAIN_API_KEY"] = ""  #Add your LANGCHAIN API Key
try:
    import langsmith
    from langsmith import traceable
    LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
    if LANGCHAIN_API_KEY:
        os.environ["LANGCHAIN_TRACING_V2"] = "true"
        os.environ["LANGCHAIN_PROJECT"] = "physics-teacher-bot"
        langsmith_client = langsmith.Client()
        LANGCHAIN_AVAILABLE = True
        print("✅ LangSmith tracing initialized successfully!")
    else:
        print("⚠️  LANGCHAIN_API_KEY not found. LangSmith tracing will be disabled.")
        langsmith_client = None
        LANGCHAIN_AVAILABLE = False
except ImportError:
    print("⚠️  LangSmith not installed. Tracing will be disabled.")
    print("💡 Run: pip install langsmith")
    langsmith_client = None
    LANGCHAIN_AVAILABLE = False
except Exception as e:
    print(f"⚠️  LangSmith initialization failed: {e}")
    langsmith_client = None
    LANGCHAIN_AVAILABLE = False

✅ LangSmith tracing initialized successfully!


In [ ]:
# Initialize Tavily client with error handling
os.environ["TAVILY_API_KEY"] = ""  #Add your TAVILY API Key
try:
    from tavily import TavilyClient
    tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    TAVILY_AVAILABLE = True
except ImportError:
    TAVILY_AVAILABLE = False
    print("⚠️  Tavily not installed. Web search functionality will be disabled.")
    print("💡 Run: pip install tavily-python")

In [ ]:
# Initialize NVIDIA NeMo Guardrails with error handling
try:
    from nemoguardrails import RailsConfig, LLMRails
    from nemoguardrails.actions import action
    NEMO_GUARDRAILS_AVAILABLE = True
except ImportError:
    NEMO_GUARDRAILS_AVAILABLE = False
    print("⚠️  NVIDIA NeMo Guardrails not installed. Safety features will be disabled.")
    print("💡 Run: pip install nemoguardrails")

In [ ]:
# Initialize ChromaDB
chroma_client = chromadb.Client()
pdf_collection = chroma_client.create_collection(name="PhysicsPDFs")
memory_collection = chroma_client.create_collection(name="ConversationMemory")

In [ ]:
# Cell: allow sync generate inside notebook event loop
import nest_asyncio
nest_asyncio.apply()
print("✅ Enabled nested asyncio; sync guardrails.generate() is now allowed.")

✅ Enabled nested asyncio; sync guardrails.generate() is now allowed.


In [ ]:
# Initialize NeMo Guardrails
from nemoguardrails import LLMRails, RailsConfig

assert "OPENAI_API_KEY" in os.environ, "Set OPENAI_API_KEY in environment."

# Load config from file
config = RailsConfig.from_path("rails")
#print("Models in config:", config.as_dict().get("models"))  # sanity check
nemo_rails = LLMRails(config)
print("✅ NeMo Guardrails initialized from rails/")

def apply_guardrails(user_message: str, bot_response: str = ""):
    """
    Apply NeMo Guardrails for safety and content filtering.
    Returns (allowed: bool, message: str).
    - If blocked, message is the guardrails response.
    - If allowed, message is either the original bot_response or 'OK'.
    """
    if not NEMO_GUARDRAILS_AVAILABLE:
        # fallback: keyword safety check
        return apply_keyword_safety_check(user_message, bot_response)

    try:
        # Use NeMo Guardrails for safety checking
        guarded_reply = nemo_rails.generate(
            messages=[{"role": "user", "content": user_message}]
        )
        # Extract text safely
        guarded_text = _extract_text_from_reply(guarded_reply)

        # If guardrails produced a block message, return blocked
        if "I can’t help with harmful, dangerous, or illegal topics." in guarded_text:
            return False, guarded_text
        if "I’m a physics tutor. Please ask science or education topics." in guarded_text:
            return False, guarded_text

        # Otherwise, if bot_response was passed, return it; else 'OK'
        return True, bot_response or "OK"

    except Exception as e:
        print(f"Guardrails error: {e}")
        # Fallback to keyword-based safety check
        return apply_keyword_safety_check(user_message, bot_response)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.onnx:   0%|          | 0.00/90.4M [00:00<?, ?B/s]

✅ NeMo Guardrails initialized from rails/


In [ ]:
class TokenUsageTracker:
    """Track token usage across different components"""

    def __init__(self):
        self.total_tokens = 0
        self.prompt_tokens = 0
        self.completion_tokens = 0
        self.costs = 0.0

    def update_from_response(self, response, model="gpt-3.5-turbo"):
        """Update token counts from OpenAI response"""
        if hasattr(response, 'usage'):
            usage = response.usage
            self.prompt_tokens += getattr(usage, 'prompt_tokens', 0)
            self.completion_tokens += getattr(usage, 'completion_tokens', 0)
            self.total_tokens += getattr(usage, 'total_tokens', 0)

            # Calculate cost (approximate)
            cost_rates = {
                "gpt-3.5-turbo": (0.0015, 0.002),  # $0.0015/1K input, $0.002/1K output
                "gpt-4": (0.03, 0.06),  # $0.03/1K input, $0.06/1K output
                "gpt-4-turbo": (0.01, 0.03)  # $0.01/1K input, $0.03/1K output
            }

            input_rate, output_rate = cost_rates.get(model, (0.0015, 0.002))
            self.costs += (self.prompt_tokens / 1000 * input_rate) + (self.completion_tokens / 1000 * output_rate)

    def get_summary(self):
        """Get token usage summary"""
        return {
            "total_tokens": self.total_tokens,
            "prompt_tokens": self.prompt_tokens,
            "completion_tokens": self.completion_tokens,
            "estimated_cost": round(self.costs, 4)
        }

In [ ]:
# Global token tracker
token_tracker = TokenUsageTracker()

def traceable_function(func):
    """Decorator that conditionally applies tracing based on availability"""
    if LANGCHAIN_AVAILABLE:
        return traceable(func)
    return func

In [ ]:
@traceable_function
def extract_text_from_pdf(pdf_path):
    """Extract text from PDF file"""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

In [ ]:
@traceable_function
def chunk_text(text, chunk_size=1000):
    """Split text into chunks"""
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

In [ ]:
@traceable_function
def store_pdf_in_db(pdf_path):
    """Extract, chunk and store PDF in ChromaDB"""
    try:
        text = extract_text_from_pdf(pdf_path)
        chunks = chunk_text(text)

        # Clear existing data
        pdf_collection.delete(where={"source": {"$ne": ""}})

        # Add chunks to ChromaDB
        for i, chunk in enumerate(chunks):
            pdf_collection.add(
                documents=[chunk],
                metadatas=[{"source": pdf_path, "chunk_id": i, "type": "pdf"}],
                ids=[f"pdf_chunk_{i}_{os.path.basename(pdf_path)}"]
            )

        # Log to LangSmith
        if langsmith_client:
            try:
                langsmith_client.create_feedback(
                    run_id=None,  # Will be associated with current trace
                    key="pdf_processing",
                    score=1.0,
                    comment=f"Processed {len(chunks)} chunks from {pdf_path}"
                )
            except Exception as e:
                print(f"LangSmith logging error: {e}")

        return f"✅ Successfully loaded {len(chunks)} chunks from {pdf_path}"
    except Exception as e:
        return f"❌ Error: {str(e)}"

In [ ]:
@traceable_function
def get_relevant_context(query, n_results=3):
    """Retrieve relevant context from PDF knowledge base"""
    try:
        results = pdf_collection.query(
            query_texts=[query],
            n_results=n_results
        )
        return "\n\n".join(results['documents'][0]) if results['documents'] else ""
    except Exception as e:
        print(f"PDF context query error: {e}")
        return ""

In [ ]:
@traceable_function
def store_conversation_memory(user_message, bot_response, conversation_context=""):
    """Store conversation in long-term memory"""
    try:
        # Create a memory entry
        memory_id = str(uuid.uuid4())
        timestamp = datetime.now().isoformat()

        # Combine user message and bot response for semantic search
        memory_text = f"User: {user_message}\nBot: {bot_response}"

        if conversation_context:
            memory_text = f"Context: {conversation_context}\n{memory_text}"

        # Store in memory collection
        memory_collection.add(
            documents=[memory_text],
            metadatas=[{
                "type": "conversation",
                "timestamp": timestamp,
                "user_message": user_message,
                "bot_response": bot_response,
                "conversation_context": conversation_context
            }],
            ids=[memory_id]
        )

        return True
    except Exception as e:
        print(f"Memory storage error: {e}")
        return False

In [ ]:
@traceable_function
def get_relevant_memories(query, n_results=2):
    """Retrieve relevant past conversations from memory"""
    try:
        results = memory_collection.query(
            query_texts=[query],
            n_results=n_results
        )

        memories = []
        if results['documents']:
            for i, doc in enumerate(results['documents'][0]):
                metadata = results['metadatas'][0][i]
                memory_text = f"Previous conversation ({metadata['timestamp'][:10]}): {doc}"
                memories.append(memory_text)

        return "\n\n".join(memories) if memories else ""
    except Exception as e:
        print(f"Memory retrieval error: {e}")
        return ""

In [ ]:
@traceable_function
def get_conversation_summary(history):
    """Generate a summary of the current conversation context"""
    if not history:
        return ""

    # Take last few exchanges for context
    recent_history = history[-3:] if len(history) > 3 else history
    context_parts = []

    for exchange in recent_history:
        if isinstance(exchange, tuple) and len(exchange) == 2:
            user_msg, bot_msg = exchange
            context_parts.append(f"User: {user_msg}")
            context_parts.append(f"Bot: {bot_msg}")
        elif isinstance(exchange, list) and len(exchange) == 2:
            user_msg, bot_msg = exchange
            context_parts.append(f"User: {user_msg}")
            context_parts.append(f"Bot: {bot_msg}")

    return "\n".join(context_parts)

In [ ]:
@traceable_function
def is_pdf_context_relevant(pdf_context, query):
    """Check if PDF context is actually relevant to the query"""
    if not pdf_context or len(pdf_context.strip()) < 50:
        return False

    # Check for common irrelevant patterns in PDF context
    irrelevant_patterns = [
        "table of contents", "copyright", "abstract", "references",
        "appendix", "index", "acknowledg", "preface"
    ]

    pdf_lower = pdf_context.lower()
    query_lower = query.lower()

    # If PDF context contains many irrelevant sections, it's likely not useful
    irrelevant_count = sum(1 for pattern in irrelevant_patterns if pattern in pdf_lower)
    if irrelevant_count > 2:
        return False

    # Check if query terms actually appear in the context
    query_terms = query_lower.split()
    relevant_terms = sum(1 for term in query_terms if len(term) > 3 and term in pdf_lower)

    # If less than 30% of meaningful query terms are in context, consider it irrelevant
    meaningful_terms = [term for term in query_terms if len(term) > 3]
    if meaningful_terms and (relevant_terms / len(meaningful_terms)) < 0.3:
        return False

    return True

In [ ]:
@traceable_function
def search_with_tavily(query):
    """Search using Tavily and get content from first result"""
    if not TAVILY_AVAILABLE:
        return "Web search unavailable. Please install tavily-python: pip install tavily-python"

    try:
        # Search for the query
        search_response = tavily_client.search(
            query=query,
            search_depth="basic",
            max_results=3
        )

        if not search_response.get('results'):
            return "No search results found."

        # Get the first result
        first_result = search_response['results'][0]
        first_url = first_result.get('url', '')

        # Use Tavily's content if available
        content = first_result.get('content', '')

        if not content:
            content = f"Title: {first_result.get('title', 'N/A')}\nURL: {first_url}"

        # Check if it's a PDF
        if first_url.lower().endswith('.pdf'):
            return f"📄 PDF Source: {first_url}\n\nThis appears to be a PDF document. Please visit the link to view the PDF: {first_url}"

        return f"🔍 Search Result from: {first_url}\n\n{content}"

    except Exception as e:
        return f"❌ Search error: {str(e)}"

In [ ]:
@traceable_function
def apply_keyword_safety_check(user_message, bot_response):
    """Fallback keyword-based safety check"""
    harmful_keywords = [
        "bomb", "weapon", "explosive", "harmful", "dangerous", "illegal",
        "hack", "cheat", "bypass", "malicious", "violence", "hurt", "kill"
    ]

    user_lower = user_message.lower()
    response_lower = bot_response.lower()

    # Check for harmful content in user message
    if any(keyword in user_lower for keyword in harmful_keywords):
        safety_message = "I apologize, but I cannot engage with that type of question. I'm here to help with physics education and related scientific topics."
        return safety_message, "🛡️ Input blocked by safety check"

    # Check for harmful content in bot response
    if any(keyword in response_lower for keyword in harmful_keywords):
        safety_message = "I need to provide a safe and educational response. Let me rephrase that to focus on the educational aspects of physics."
        return safety_message, "🛡️ Response moderated by safety check"

    return bot_response, "🛡️ Safety check passed"

In [ ]:
# Cell: generator helper using nemo_rails + prompt_mgr

def _extract_text_from_reply(reply):
    if reply is None:
        return ""
    if isinstance(reply, str):
        return reply
    if isinstance(reply, dict):
        if "content" in reply and isinstance(reply["content"], str):
            return reply["content"]
        if "message" in reply and isinstance(reply["message"], str):
            return reply["message"]
        choices = reply.get("choices") or reply.get("outputs") or None
        if choices and isinstance(choices, (list, tuple)) and len(choices) > 0:
            first = choices[0]
            if isinstance(first, dict):
                msg = first.get("message") or first.get("content") or first.get("text")
                if isinstance(msg, dict):
                    return msg.get("content") or msg.get("text") or ""
                if isinstance(msg, str):
                    return msg
            elif isinstance(first, str):
                return first
        return str(reply.get("content") or reply.get("message") or "")
    if isinstance(reply, (list, tuple)) and len(reply) > 0:
        return _extract_text_from_reply(reply[0])
    return str(reply)

def _extract_text_from_reply1(reply):
    if reply is None:
        return ""
    if isinstance(reply, str):
        return reply
    if isinstance(reply, dict):
        # common keys: 'content', 'message', 'text', 'choices'
        if "content" in reply and isinstance(reply["content"], str):
            return reply["content"]
        if "message" in reply and isinstance(reply["message"], str):
            return reply["message"]
        # NeMo-like: maybe reply is {'choices': [{'message': {'content': '...'}}]}
        choices = reply.get("choices") or reply.get("outputs") or None
        if choices and isinstance(choices, (list, tuple)) and len(choices) > 0:
            first = choices[0]
            if isinstance(first, dict):
                # try nested message.content
                msg = first.get("message") or first.get("content") or first.get("text")
                if isinstance(msg, dict):
                    return msg.get("content") or msg.get("text") or ""
                if isinstance(msg, str):
                    return msg
            elif isinstance(first, str):
                return first
        # fallback: stringify small parts
        return str(reply.get("content") or reply.get("message") or "")
    # lists/tuples
    if isinstance(reply, (list, tuple)) and len(reply) > 0:
        return _extract_text_from_reply(reply[0])
    return str(reply)


def answer_query_with_nemo(user_message: str, memory_summary: str = "", diagnostics: dict = None, clarify: bool = False):
    """
    Calls nemo_rails.generate with system + user messages rendered from prompt_mgr.
    Returns (assistant_text: str, raw_reply: Any)
    """
    system_text = prompt_mgr.render_system()
    user_text = prompt_mgr.render_user(user_message, clarify=clarify, memory_summary=memory_summary, diagnostics=diagnostics)

    messages = [
        {"role": "system", "content": system_text},
        {"role": "user", "content": user_text},
    ]

    try:
        reply = nemo_rails.generate(messages=messages)
    except Exception as e:
        # fallback: return error-like message and the exception for logging
        return f"Error generating response: {e}", {"error": str(e)}

    assistant_text = _extract_text_from_reply(reply)
    return assistant_text, reply

In [ ]:
ans, raw, src = generate_answer("what is resistance in current electricity?", memory_summary="", diagnostics={}, clarify=False)
print("Answer:", ans)
print("Source:", src)
print("Raw type:", type(raw))

=== MODEL MESSAGES ===
system : You are a concise, patient physics tutor for high-school and early-college students.
user : what is resistance in current electricity?
Answer: Resistance in current electricity is a measure of how much a material or component opposes the flow of electric current. It is denoted by the symbol \( R \) and is measured in ohms (\( \Omega \)). The higher the resistance, the more difficult it is for current to flow through a circuit. Resistance is determined by factors such as the material of the conductor, its length, cross-sectional area, and temperature. Ohm's Law states that the current flowing through a conductor is directly proportional to the voltage across it and inversely proportional to the resistance of the conductor: \( I = \frac{V}{R} \), where \( I \) is the current, \( V \) is the voltage, and \( R \) is the resistance.
Source: nemo
Raw type: <class 'dict'>


In [ ]:
# Diagnostics CSV logger
import csv
from pathlib import Path
from datetime import datetime

LOGS_DIR = Path("logs")
LOGS_DIR.mkdir(exist_ok=True)
DIAG_CSV = LOGS_DIR / "diagnostics.csv"

# ensure header exists
if not DIAG_CSV.exists():
    with DIAG_CSV.open("w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([
            "timestamp", "user_message", "allowed", "clarify", "generator",
            "edu_topk_mean", "edu_max", "harm_topk_mean", "harm_max",
            "len_tokens", "answer_snippet", "raw_reply_summary"
        ])

def _safe_get(d: dict, key: str, default=None):
    try:
        return d.get(key, default) if isinstance(d, dict) else default
    except Exception:
        return default

def log_diagnostics_csv(user_message: str, allowed: bool, clarify: bool, generator: str,
                        diagnostics: dict, answer_text: str, raw_reply: object):
    """
    Append a single diagnostics row. Non-blocking: exceptions are swallowed.
    Keep answer_snippet short to avoid huge CSV cells.
    """
    try:
        ts = datetime.utcnow().isoformat() + "Z"
        edu_topk = _safe_get(diagnostics, "edu_topk_mean", "")
        edu_max = _safe_get(diagnostics, "edu_max", "")
        harm_topk = _safe_get(diagnostics, "harm_topk_mean", "")
        harm_max = _safe_get(diagnostics, "harm_max", "")
        length = _safe_get(diagnostics, "len_tokens", "")
        snippet = (answer_text or "")[:400].replace("\n", " ").strip()
        # raw_reply_summary: try to capture role/content keys if present
        raw_summary = ""
        try:
            if isinstance(raw_reply, dict):
                raw_summary = raw_reply.get("content") or str({k: raw_reply.get(k) for k in ("role","type") if k in raw_reply})
            elif isinstance(raw_reply, (list, tuple)):
                # summarize first element
                first = raw_reply[0]
                if isinstance(first, dict):
                    raw_summary = first.get("content", "")[:200]
                else:
                    raw_summary = str(first)[:200]
            else:
                raw_summary = str(raw_reply)[:200]
        except Exception:
            raw_summary = ""
        row = [
            ts, user_message, str(bool(allowed)), str(bool(clarify)), generator,
            edu_topk, edu_max, harm_topk, harm_max, length, snippet, raw_summary
        ]
        with DIAG_CSV.open("a", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(row)
    except Exception:
        # never raise from logger
        pass

print("✅ diagnostics CSV logger ready:", DIAG_CSV)


✅ diagnostics CSV logger ready: logs/diagnostics.csv


In [ ]:
def rewrite_ambiguous_query_with_llm(user_message: str, history: list, max_history_turns: int = 4):
    """
    Ask the LLM to rewrite a short/ambiguous user_message into an explicit query
    using the last few turns. Returns the rewritten string or the original message.
    """
    # Build a short prompt that is safe and deterministic
    recent = []
    for turn in history[-max_history_turns:]:
        if isinstance(turn, (list, tuple)) and len(turn) >= 2:
            recent.append(f"User: {turn[0]}\nAssistant: {turn[1]}")
    context_block = "\n\n".join(recent)

    rewrite_prompt = (
        "You are a concise assistant that rewrites short or pronoun-based follow-up "
        "questions into explicit, self-contained questions using the conversation history. "
        "Do not add new facts. If the user message is already explicit, return it unchanged.\n\n"
        f"Conversation history:\n{context_block}\n\n"
        f"User follow-up: {user_message}\n\n"
        "Return a single-line rewritten question that is explicit (for example: "
        "'dimensions of resistance and resistivity')."
    )

    # Call your generator in a lightweight mode; prefer the same generator you use for answers
    rewritten, raw, src = generate_answer(rewrite_prompt, history=[], clarify=False, prefer_generator="nemo")
    # Basic safety: if rewrite looks like a template or is empty, keep original
    if not rewritten or len(rewritten.strip()) < 3 or "{user_message}" in rewritten:
        return user_message
    return rewritten.strip()

In [ ]:
# Top-level handler that vets and generates

def handle_user_message(user_message: str, history=None, prefer_generator: str = None):
    """
    Top-level handler:
    - RAG-first: try PDF -> memory -> web
    - If RAG yields a confident context, generate immediately using that context + history
    - Otherwise run vet_query; if clarify==False, generate using history and diagnostics
    - If clarify==True, return a user-facing clarification payload
    """
    # Ensure history is a list
    history = history or []
    print("handle_user_message history len:", len(history or []), "history:", history[-6:])

    user_message = rewrite_ambiguous_query_with_llm(user_message, history)

    # 1) Try RAG: PDF context and memory context
    pdf_context = get_relevant_context(user_message, n_results=3) or ""
    memory_context = get_relevant_memories(user_message, n_results=2) or ""

    use_pdf = is_pdf_context_relevant(pdf_context, user_message)

    # 2) If PDF is useful, generate immediately using augmented context + history
    if use_pdf:
        augmented_message = f"Context:\n{pdf_context}\n\nUser question: {user_message}"
        answer_text, raw_reply, source = generate_answer(
            augmented_message,
            memory_summary=memory_context,
            diagnostics={},
            clarify=False,
            history=history,
            pdf_context=pdf_context,
            prefer_generator=prefer_generator
        )
        return {
            "allowed": True,
            "clarify": False,
            "answer": answer_text,
            "raw_reply": raw_reply,
            "diagnostics": {},
            "memory_summary": memory_context,
            "generator": source,
        }

    # 3) No confident RAG hit: vet the query (guardrails + educational filter)
    ok, payload = vet_query(user_message)
    if not ok:
        return {"allowed": False, "answer": None, "reason": payload.get("reason")}

    memory_summary = payload.get("memory_summary", "")
    diagnostics = payload.get("diagnostics", {})
    clarify = payload.get("clarify", False)

    # 4) If clarify requested, return user-facing clarification payload
    if clarify:
        clarif_text = prompt_mgr.render_clarify_user(
            original_query=user_message,
            memory_summary=memory_summary,
            diagnostics=diagnostics
        )
        clarif_payload = {
            "title": "Quick clarification",
            "preview": clarif_text.split("\n")[0],
            "body": clarif_text,
            "quick_replies": [
                "Electrical resistance in circuits.",
                "Air resistance in motion.",
                "Give a short definition with one circuit example."
            ],
            "diagnostics": diagnostics or {},
            "memory_summary": memory_summary or ""
        }
        # log diagnostics if you want
        try:
            log_diagnostics_csv(
                user_message=user_message, allowed=True, clarify=True,
                generator="clarify", diagnostics=diagnostics,
                answer_text=clarif_payload.get("preview", ""),
                raw_reply={"clarification_payload": clarif_payload}
            )
        except Exception:
            pass

        return {
            "allowed": True,
            "clarify": True,
            "clarification_payload": clarif_payload,
            "diagnostics": diagnostics,
            "memory_summary": memory_summary,
            "generator": "clarify",
        }

    # 5) Otherwise generate an answer using history + memory + diagnostics
    answer_text, raw_reply, source = generate_answer(
        user_message,
        memory_summary=memory_summary,
        diagnostics=diagnostics,
        clarify=False,
        history=history,
        pdf_context=pdf_context,
        prefer_generator=prefer_generator
    )

    # store memory and log diagnostics as appropriate
    try:
        store_conversation_memory(user_message, answer_text, conversation_context=memory_summary)
    except Exception:
        pass

    try:
        log_diagnostics_csv(
            user_message=user_message, allowed=True, clarify=False,
            generator=source, diagnostics=diagnostics,
            answer_text=(answer_text or "")[:400], raw_reply=raw_reply
        )
    except Exception:
        pass

    return {
        "allowed": True,
        "clarify": False,
        "answer": answer_text,
        "raw_reply": raw_reply,
        "diagnostics": diagnostics,
        "memory_summary": memory_summary,
        "generator": source,
    }

def handle_user_message1(user_message: str, prefer_generator: str = None):
    pdf_context = get_relevant_context(user_message, n_results=3)  # existing function
    memory_context = get_relevant_memories(user_message, n_results=2)  # existing function

    # 2) Decide if PDF context is actually useful
    use_pdf = is_pdf_context_relevant(pdf_context, user_message)

    # If PDF is useful, prefer it and generate answer immediately using that context
    if use_pdf:
        # Prepend the PDF context to the user message so the prompt_mgr includes it
        augmented_message = f"Context:\n{pdf_context}\n\nUser question: {user_message}"
        answer_text, raw_reply, source = generate_answer(
            user_message=augmented_message,
            memory_summary=memory_summary,
            diagnostics={},  # or pass diagnostics if available
            clarify=False
        )
        # store memory, log diagnostics as you already do
        return {
            "allowed": True,
            "clarify": False,
            "answer": answer_text,
            "raw_reply": raw_reply,
            "diagnostics": {},
            "memory_summary": memory_summary,
            "generator": source,
        }
    ok, payload = vet_query(user_message)
    if not ok:
        return {"allowed": False, "answer": None, "reason": payload.get("reason")}

    memory_summary = payload.get("memory_summary", "")
    diagnostics = payload.get("diagnostics", {})
    clarify = payload.get("clarify", False)

    # If clarify is True, return the clarification prompt instead of generating a full answer
    if clarify:
        clarif_text = prompt_mgr.render_clarify_user(
            original_query=user_message,
            memory_summary=memory_summary,
            diagnostics=diagnostics
        )
        clarif_payload = {
            "title": "Quick clarification",
            "preview": clarif_text.split("\n")[0],
            "body": clarif_text,
            "quick_replies": [
                "Electrical resistance in circuits.",
                "Air resistance in motion.",
                "Give a short definition with one circuit example."
            ],
            "diagnostics": diagnostics or {},
            "memory_summary": memory_summary or ""
        }

        try:
            log_diagnostics_csv(
                user_message=user_message, allowed=True, clarify=True,
                generator="clarify", diagnostics=diagnostics,
                answer_text=clarif_payload.get("preview", ""),
                raw_reply={"clarification_payload": clarif_payload}
            )
        except Exception:
            pass

        return {
            "allowed": True,
            "clarify": True,
            "clarification_payload": clarif_payload,
            "diagnostics": diagnostics,
            "memory_summary": memory_summary,
            "generator": "clarify",
        }

    # Otherwise generate a full answer as before
    answer_text, raw_reply, source = generate_answer(user_message, memory_summary=memory_summary, diagnostics=diagnostics, clarify=False)
    return {"allowed": True, "clarify": False, "answer": answer_text, "raw_reply": raw_reply, "generator": source}

# quick smoke test (run after adding cell)
#print(handle_user_message("What is resistance in a circuit?"))

In [ ]:
# Cell: accept clarification and generate final answer (new cell)
def accept_clarification_and_generate(clarification_text: str, original_user_message: str = None,
                                      diagnostics: dict = None, memory_summary: str = "", prefer_generator: str = None):
    """
    Treat clarification_text as the user's confirmed/expanded message and generate a final answer.
    - clarification_text: the text the user confirmed or selected from quick replies
    - original_user_message: optional, for logging/context
    - diagnostics, memory_summary: optional, passed through from vet_query
    Returns the same structure as handle_user_message but with clarify=False and final answer.
    """
    # Optionally log that the user confirmed a clarification (best-effort)
    try:
        log_diagnostics_csv(user_message=original_user_message or clarification_text,
                            allowed=True, clarify=False, generator="clarify_confirm",
                            diagnostics=diagnostics or {}, answer_text=clarification_text, raw_reply={})
    except Exception:
        pass

    # Now generate the final answer using the chosen generator
    answer_text, raw_reply, source = generate_answer(
        user_message=clarification_text,
        memory_summary=memory_summary,
        diagnostics=diagnostics or {},
        clarify=False,
        prefer=prefer_generator
    )

    # store interaction in memory (best-effort)
    try:
        memory.store_interaction(user_message=clarification_text, assistant_reply=answer_text, diagnostics=diagnostics or {})
    except Exception:
        pass

    # log the final generation
    try:
        log_diagnostics_csv(user_message=clarification_text, allowed=True, clarify=False,
                            generator=source, diagnostics=diagnostics or {},
                            answer_text=answer_text, raw_reply=raw_reply)
    except Exception:
        pass

    return {
        "allowed": True,
        "clarify": False,
        "answer": answer_text,
        "raw_reply": raw_reply,
        "diagnostics": diagnostics or {},
        "memory_summary": memory_summary,
        "generator": source,
    }


In [ ]:
# Tweak these values to change when clarification is requested

CLARIFY_CONFIG = {
    "max_short_tokens": 6,        # queries with <= this token count are considered "short"
    "edu_lower": 0.30,            # lower bound of edu_topk_mean to consider borderline
    "edu_upper": 0.38,            # upper bound of edu_topk_mean for borderline
    "force_clarify_on_empty": True,  # clarify if diagnostics missing or empty for short queries
}

def decide_clarify(diagnostics: dict, len_tokens: int) -> bool:
    """
    Return True only when diagnostics indicate borderline educational intent
    or explicit ambiguity. Do not force clarify on empty diagnostics.
    """
    if not isinstance(diagnostics, dict) or not diagnostics:
        # Do not force clarification just because diagnostics is empty
        return False

    edu_score = float(diagnostics.get("edu_topk_mean", 0.0))
    # short query borderline range -> clarify
    if len_tokens <= CLARIFY_CONFIG["max_short_tokens"] and CLARIFY_CONFIG["edu_lower"] <= edu_score < CLARIFY_CONFIG["edu_upper"]:
        return True

    # Add other rules if needed (e.g., multiple high-scoring conflicting contexts)
    return False

In [ ]:
@traceable_function
def apply_safety_guardrails(user_message, bot_response):
    """Apply NeMo Guardrails for safety and content filtering"""
    if not NEMO_GUARDRAILS_AVAILABLE:
        return bot_response, "⚠️ Guardrails disabled"

    try:
        # Use NeMo Guardrails for safety checking
        guarded_response = nemo_rails.generate(
            messages=[{"role": "user", "content": user_message}]
        )

        guarded_response = _extract_text_from_reply(guarded_response)

        # If guardrails modified the response, use the guarded version
        if guarded_response and guarded_response != bot_response:
            return guarded_response, "🛡️ Response moderated by guardrails"

        return bot_response, "🛡️ Safety check passed"

    except Exception as e:
        print(f"Guardrails error: {e}")
        # Fallback to keyword-based safety check
        return apply_keyword_safety_check(user_message, bot_response)

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
import re
import numpy as np
from functools import lru_cache

# Model and precomputed embeddings (keep as you already have)
model = SentenceTransformer("all-MiniLM-L6-v2")

harmful_phrases = [
    "harmful", "dangerous", "illegal", "weapon", "explosive",
    "hack", "cheat", "bypass", "circumvent", "malicious"
]
educational_phrases = [
    "physics", "quantum", "mechanics", "thermodynamics", "electromagnetism",
    "resistance", "current", "voltage", "circuit", "ohm", "electricity",
    "relativity", "optics", "atomic", "nuclear", "particle", "force",
    "energy", "motion", "velocity", "acceleration", "gravity", "electric",
    "magnetic", "wave", "light", "sound", "heat", "temperature",
    "science", "math", "chemistry", "biology", "astronomy", "engineering",
    "technology", "computer", "programming", "learn", "study", "education",
    "explain", "how does", "what is", "why does"
]

# Precompute embeddings (ensure 2-D tensors and normalize)
harmful_embeds = model.encode(harmful_phrases, convert_to_tensor=True)  # shape (N_h, D)
educational_embeds = model.encode(educational_phrases, convert_to_tensor=True)  # shape (N_e, D)

# Normalize rows (works for both 1-D and 2-D safely)
def normalize_rows(t: torch.Tensor) -> torch.Tensor:
    if t.dim() == 1:
        t = t.unsqueeze(0)            # make (1, D)
        t = util.normalize_embeddings(t)
        return t.squeeze(0)          # back to (D,)
    else:
        return util.normalize_embeddings(t)  # (N, D)

harmful_embeds = normalize_rows(harmful_embeds)        # (N_h, D)
educational_embeds = normalize_rows(educational_embeds)  # (N_e, D)

# Simple negation detector used by check_educational_context
_negation_re = re.compile(
    r"\b(no|not|never|don't|dont|cannot|can't|cant|without|avoid|avoidance)\b",
    flags=re.IGNORECASE,
)

@lru_cache(maxsize=4096)
def _embed_text_cached(text: str) -> torch.Tensor:
    emb = model.encode(text, convert_to_tensor=True)  # (D,)
    # keep user embedding as 1-D for cos_sim compatibility
    if emb.dim() == 2 and emb.shape[0] == 1:
        emb = emb.squeeze(0)
    # normalize 1-D vector safely
    emb = normalize_rows(emb)  # returns 1-D
    return emb

@traceable_function
def check_educational_context(
    user_message: str,
    harmful_threshold: float = 0.45,
    educational_threshold: float = 0.35,
    top_k: int = 3,
    keyword_fallback_threshold: int = 1
):
    """
    Returns (allowed: bool, reason: str, diagnostics: dict)
    diagnostics contains scores for tuning and logging.
    """

    text = user_message.strip()
    if not text:
        return False, "Empty query"

    # Quick keyword fallback for very short queries
    tokens = re.findall(r"\w+", text.lower())
    if len(tokens) <= 3:
        # count educational keywords present (simple fallback)
        edu_hits = sum(1 for kw in educational_phrases if kw in text.lower())
        harm_hits = sum(1 for kw in harmful_phrases if kw in text.lower())
        if harm_hits >= keyword_fallback_threshold and not _negation_re.search(text):
            return False, "This short query contains harmful keywords"
        if edu_hits >= keyword_fallback_threshold:
            return True, "OK"
        # fall through to embedding check for ambiguous short queries

    # Embedding-based checks
    user_emb = _embed_text_cached(text)  # normalized

    # compute cosine similarities to each set
    harm_sims = util.cos_sim(user_emb, harmful_embeds).cpu().numpy().flatten()
    edu_sims = util.cos_sim(user_emb, educational_embeds).cpu().numpy().flatten()

    # ensure harm_sims and edu_sims are numpy arrays
    harm_sims = np.asarray(harm_sims)
    edu_sims = np.asarray(edu_sims)

    # top-k mean using numpy
    harm_topk_mean = float(np.sort(harm_sims)[-top_k:].mean())
    harm_max = float(harm_sims.max())
    edu_topk_mean = float(np.sort(edu_sims)[-top_k:].mean())
    edu_max = float(edu_sims.max())

    diagnostics = {
        "harm_topk_mean": harm_topk_mean,
        "harm_max": harm_max,
        "edu_topk_mean": edu_topk_mean,
        "edu_max": edu_max,
        "len_tokens": len(tokens)
    }

    # Harmful detection: prefer topk mean but also consider max
    if (harm_topk_mean >= harmful_threshold or harm_max >= harmful_threshold + 0.05) and not _negation_re.search(text):
        return False, "This query seems harmful or unsafe."

    # Educational detection
    if edu_topk_mean >= educational_threshold or edu_max >= educational_threshold + 0.05:
        return True, "OK"

    # Default: block with helpful message
    return False, "Please ask questions related to physics or education."

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Prompt loader and LangChain templates
import yaml
from pathlib import Path
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

TEMPLATES_PATH = Path("prompts/templates.yml")

with open(TEMPLATES_PATH, "r", encoding="utf-8") as f:
    raw = yaml.safe_load(f)

# Build SystemMessagePromptTemplate
system_tpl = raw.get("system", {}).get("tutor", {}).get("content", "")
system_msg_template = SystemMessagePromptTemplate.from_template(system_tpl)

# Build HumanMessagePromptTemplate variants
user_default_tpl = raw.get("user", {}).get("default", {}).get("content", "")
user_clarify_tpl = raw.get("user", {}).get("clarification", {}).get("content", "")

human_default = HumanMessagePromptTemplate.from_template(user_default_tpl)
human_clarify = HumanMessagePromptTemplate.from_template(user_clarify_tpl)

# Compose a ChatPromptTemplate (example: system + human default)
chat_prompt_default = ChatPromptTemplate.from_messages([system_msg_template, human_default])

In [ ]:
class _MemoryStub:
    def get_summary_for_user(self, *args, **kwargs):
        # return empty string or a short default summary
        return ""

    def store_interaction(self, *args, **kwargs):
        return True

# expose as `memory` so existing code works unchanged
memory = _MemoryStub()


In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

def vet_query(user_message: str):
    ok, msg = apply_guardrails(user_message)
    if not ok:
        return False, {"reason": msg}

    result = check_educational_context(user_message)
    # support both 2-tuple and 3-tuple returns
    if isinstance(result, tuple) and len(result) == 3:
        allow, reason, diagnostics = result
    elif isinstance(result, tuple) and len(result) == 2:
        allow, reason = result
        diagnostics = {}
    else:
        raise ValueError("Unexpected return from check_educational_context")

    logging.info("context_check: allow=%s reason=%s diagnostics=%s", allow, reason, diagnostics)

    if not allow:
        return False, {"reason": reason, "diagnostics": diagnostics}

    # Lightweight RAG check (safe to call here)
    pdf_context = ""
    try:
        pdf_context = get_relevant_context(user_message, n_results=3)
    except Exception:
        pdf_context = ""

    # If PDF context exists and is relevant, do not force clarification here
    if pdf_context and is_pdf_context_relevant(pdf_context, user_message):
        # Let the caller handle generating with PDF context (RAG-first)
        return True, {"memory_summary": memory.get_summary_for_user() if hasattr(memory, "get_summary_for_user") else "", "diagnostics": diagnostics, "clarify": False}

    # Now decide clarify using the diagnostics but do NOT force clarify just because diagnostics is empty
    # (only clarify when diagnostics indicate borderline educational intent or explicit ambiguity)
    clarify = decide_clarify(diagnostics, diagnostics.get("len_tokens", 0))

    # If diagnostics is empty and RAG returned nothing, prefer to generate a short answer rather than clarify
    if (not diagnostics) and (not pdf_context):
        clarify = False

    # Return as before
    return True, {"memory_summary": memory.get_summary_for_user() if hasattr(memory, "get_summary_for_user") else "", "diagnostics": diagnostics, "clarify": clarify}


In [ ]:
def format_clarification_for_ui(user_message: str, memory_summary: str = "", diagnostics: dict = None, max_preview_chars: int = 240):
    body = prompt_mgr.render_clarify_user(
        original_query=user_message,
        memory_summary=memory_summary,
        diagnostics=diagnostics
    )
    preview = body.strip().split("\n\n")[0]
    if len(preview) > max_preview_chars:
        preview = preview[:max_preview_chars].rsplit(" ", 1)[0] + "…"

    quick_replies = [
        "Electrical resistance in circuits.",
        "Air resistance in motion.",
        "Give a short definition with one circuit example."
    ]

    return {
        "title": "Quick clarification",
        "preview": preview,
        "body": body,
        "quick_replies": quick_replies
    }

In [ ]:
# Clarification payload helper and optional Flask endpoint
from flask import Flask, request, jsonify  # optional; only needed if you enable the endpoint

def get_clarification_payload(user_message: str, memory_summary: str = "", diagnostics: dict = None):
    """
    Return a JSON-serializable dict with the clarification prompt and UI metadata.
    Use this from your frontend or call it inside your notebook.
    """
    # render the clarification prompt using prompt_mgr
    body = prompt_mgr.render_clarify_user(
    original_query=user_message,
    memory_summary=memory_summary,
    diagnostics=diagnostics
)
    preview = body.strip().split("\n\n")[0]
    quick_replies = [
        "Can you give more context or an example?",
        "I mean the circuit in my homework, not a theoretical example.",
        "Show a short definition and one example."
    ]
    return {
        "title": "Quick clarification",
        "preview": preview,
        "body": body,
        "quick_replies": quick_replies,
        "diagnostics": diagnostics or {},
        "memory_summary": memory_summary or ""
    }

# Optional: lightweight Flask endpoint for local testing
# Uncomment and run this cell to start a local server (only in environments that allow Flask)
# app = Flask(__name__)
#
# @app.route("/clarify", methods=["POST"])
# def clarify_endpoint():
#     payload = request.get_json(force=True)
#     user_message = payload.get("user_message", "")
#     diagnostics = payload.get("diagnostics", {})
#     memory_summary = payload.get("memory_summary", "")
#     return jsonify(get_clarification_payload(user_message, memory_summary=memory_summary, diagnostics=diagnostics))
#
# if __name__ == "__main__":
#     app.run(port=8787, debug=True)


In [ ]:
# Prompt manager wrapper
from langchain.prompts.chat import ChatPromptTemplate
from typing import Dict

# reuse the templates you already created: system_msg_template, human_default, human_clarify, chat_prompt_default
# Build a ChatPromptTemplate for clarification variant too
chat_prompt_clarify = ChatPromptTemplate.from_messages([system_msg_template, human_clarify])

class PromptManager:
    def __init__(self, default_prompt: ChatPromptTemplate, clarify_prompt: ChatPromptTemplate,
                 system_template: SystemMessagePromptTemplate = None, templates_yaml: dict = None):
        # store the LangChain prompt objects you already created
        self.default = default_prompt
        self.clarify = clarify_prompt
        # explicit system template object (fallback to the one used to build chat prompts)
        self.system_template = system_template
        # raw YAML dict for optional clarify_user template
        self.templates = templates_yaml or {}

    def render_system(self, **kwargs) -> str:
        """
        Return a single system string. Use the explicit system template if available,
        otherwise fall back to the default system prompt object.
        """
        try:
            if self.system_template:
                pv = self.system_template.format_prompt(**kwargs)
                msgs = pv.to_messages()
                return msgs[0].content if msgs else ""
            # fallback: try the default ChatPromptTemplate's system message if present
            pv = system_msg_template.format_prompt(**kwargs)
            msgs = pv.to_messages()
            return msgs[0].content if msgs else ""
        except Exception:
            # fail-safe short system prompt
            return "You are a concise, patient physics tutor for high-school and early-college students."

    def render_user(self, user_message: str, clarify: bool = False) -> str:
        """
        Render the human message. Try LangChain rendering first; if the result
        still contains an unsubstituted placeholder like {user_message},
        perform a safe replacement on the final message content.
        """
        try:
            tpl = self.clarify if clarify else self.default
            rendered = tpl.format_prompt(user_message=user_message)
            msgs = rendered.to_messages()
            if not msgs:
                return user_message

            content = msgs[-1].content or ""
            # If LangChain didn't substitute, replace common placeholders
            if "{user_message}" in content or "{input}" in content or "{text}" in content or "{query}" in content:
                content = content.replace("{user_message}", user_message)
                content = content.replace("{input}", user_message)
                content = content.replace("{text}", user_message)
                content = content.replace("{query}", user_message)
            return content
        except Exception:
            return user_message



    def render_clarify_user(self, original_query: str, memory_summary: str = "", diagnostics: dict = None) -> str:
        """
        Produce a short, user-facing clarification prompt. Prefer a YAML template 'clarify_user'
        if present; otherwise return a concise default clarification text.
        This text is for the UI only and should not be sent to the model as a system instruction.
        """
        try:
            tmpl = self.templates.get("clarify_user")
            if tmpl:
                return tmpl.format(original_query=original_query, memory_summary=memory_summary or "")
        except Exception:
            pass

        # Fallback user-facing clarification
        return (
            f'Could you clarify what you mean by "{original_query}"?\n'
            "- electrical resistance in circuits (Ohm’s law)\n"
            "- air resistance (drag)\n"
            "- something else?\n\n"
            "Please choose one or add a brief detail."
        )

# instantiate and expose
# when you create the PromptManager, pass the system template and raw YAML if available
prompt_mgr = PromptManager(
    default_prompt=chat_prompt_default,
    clarify_prompt=chat_prompt_clarify,
    system_template=system_msg_template,
    templates_yaml=raw  # the YAML dict you loaded earlier
)
print("✅ PromptManager ready")


✅ PromptManager ready


In [ ]:
ans, raw, src = generate_answer("What is resistance in current electricity?", history=[], memory_summary="", diagnostics={}, clarify=False)
print("MODEL MESSAGES (should show the real question under user)")
print("Answer snippet:", (ans or "")[:400])

=== MODEL MESSAGES ===
system : You are a concise, patient physics tutor for high-school and early-college students.
user : What is resistance in current electricity?
MODEL MESSAGES (should show the real question under user)
Answer snippet: Resistance in current electricity is a measure of how much a material or device opposes the flow of electric current. It is denoted by the symbol \( R \) and is measured in ohms (\( \Omega \)). The higher the resistance, the more difficult it is for current to flow through a circuit. Resistance can be influenced by factors such as the material of the conductor, its length, cross-sectional area, an


In [ ]:
# default preference: "nemo" or "openai"
GENERATOR_PREFERENCE = "nemo"  # change to "openai" to prefer OpenAI
OPENAI_MODEL = "gpt-3.5-turbo"

def generate_answer(
    user_message: str,
    memory_summary: str = "",
    diagnostics: dict = None,
    clarify: bool = False,
    history=None,
    pdf_context: str = "",
    prefer_generator: str = None
):
    """
    Build messages with: system, recent history (assistant/user alternating),
    optional RAG/memory context (as a single system augmentation),
    then the current user turn. Call the selected generator and extract assistant text.
    Returns (assistant_text, raw_reply, source) or ("", raw_reply, source) on error.
    """
    history = history or []
    diagnostics = diagnostics or {}
    source = None

    # 1) Render system text
    try:
        system_text = prompt_mgr.render_system()
    except Exception:
        system_text = "You are a concise, patient physics tutor for high-school and early-college students."

    messages = [{"role": "system", "content": system_text.strip()}]

    # 2) Include last N exchanges from history (sanitize)
    try:
        for turn in history[-6:]:
            if isinstance(turn, (list, tuple)):
                if len(turn) >= 1 and turn[0]:
                    u = str(turn[0]).strip()
                    if u:
                        messages.append({"role": "user", "content": u})
                if len(turn) >= 2 and turn[1]:
                    a = str(turn[1]).strip()
                    if a:
                        messages.append({"role": "assistant", "content": a})
    except Exception:
        pass

    # 3) Attach RAG/memory as a single system augmentation
    context_blocks = []
    if pdf_context and pdf_context.strip():
        context_blocks.append("PDF context:\n" + pdf_context.strip())
    if memory_summary and memory_summary.strip():
        context_blocks.append("Memory summary:\n" + memory_summary.strip())

    if context_blocks:
        messages.append({
            "role": "system",
            "content": "\n\n---\n" + "\n\n---\n".join(context_blocks)
        })

    # 4) Render user text once (no second .format)
    try:
        user_text = prompt_mgr.render_user(user_message=user_message, clarify=clarify)
    except Exception:
        user_text = user_message

    messages.append({"role": "user", "content": (user_text or user_message).strip()})

    # Optional debug (uncomment while testing)
    print("=== MODEL MESSAGES ===")
    for m in messages:
        print(m.get("role"), ":", (m.get("content") or "")[:800])
    print("======================")

    # 5) Call generator (robust preference)
    try:
        gen_pref_raw = prefer_generator if isinstance(prefer_generator, str) else GENERATOR_PREFERENCE
        gen_pref = (gen_pref_raw or "nemo").strip().lower()

        if gen_pref == "nemo":
            raw_reply = nemo_rails.generate(messages=messages)
            assistant_text = _extract_text_from_reply(raw_reply)
            source = "nemo"
        else:
            resp = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=messages)
            assistant_text = resp["choices"][0]["message"]["content"]
            raw_reply = resp
            source = "openai"
    except Exception as e:
        return "", {"error": str(e)}, None

    # 6) Retry once if greeting-like reply
    def _is_greeting(text: str) -> bool:
        t = (text or "").strip().lower()
        prefixes = ("hello", "hi", "hey", "how can i", "how may i", "greetings")
        return any(t.startswith(p) for p in prefixes)

    if assistant_text and _is_greeting(assistant_text):
        retry_messages = list(messages)
        retry_messages.append({
            "role": "user",
            "content": (user_text or user_message).strip() + "\n\nPlease answer the user's question directly."
        })
        try:
            if gen_pref == "nemo":
                raw_retry = nemo_rails.generate(messages=retry_messages)
                assistant_text_retry = _extract_text_from_reply(raw_retry)
                if assistant_text_retry and not _is_greeting(assistant_text_retry):
                    return assistant_text_retry, raw_retry, source
            else:
                resp2 = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=retry_messages)
                assistant_text_retry = resp2["choices"][0]["message"]["content"]
                if assistant_text_retry and not _is_greeting(assistant_text_retry):
                    return assistant_text_retry, resp2, source
        except Exception:
            pass

    return assistant_text or "", raw_reply, source

In [ ]:
ans, raw, src = generate_answer("Explain electrical resistance briefly and give one circuit example.", history=[], memory_summary="", diagnostics={}, clarify=False)
print("Source:", src)
print("Answer snippet:", (ans or "")[:800])

Source: nemo
Answer snippet: I'm here to help with any physics-related questions you may have. Feel free to ask, and I'll do my best to provide you with a clear and accurate answer.


In [ ]:
history = []
history.append(["What is resistance in current electricity?", gradio_adapter("What is resistance in current electricity?", history)])
history.append(["What is resistivity in current electricity?", gradio_adapter("What is resistivity in current electricity?", history)])
print("History before compare:", history)
print("Compare reply:", gradio_adapter("Compare the two as per current electricity.", history))

Adapter -> handle_user_message history: []
_pending_clarify keys: []
handle_user_message history len: 0 history: []


/tmp/ipython-input-3743950133.py:33: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + "Z"


Adapter -> handle_user_message history: [['What is resistance in current electricity?', "I'm here to help with any physics-related questions you may have. Feel free to ask, and I'll do my best to provide you with a clear and accurate answer."]]
_pending_clarify keys: []
handle_user_message history len: 1 history: [['What is resistance in current electricity?', "I'm here to help with any physics-related questions you may have. Feel free to ask, and I'll do my best to provide you with a clear and accurate answer."]]


/tmp/ipython-input-3743950133.py:33: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + "Z"


History before compare: [['What is resistance in current electricity?', "I'm here to help with any physics-related questions you may have. Feel free to ask, and I'll do my best to provide you with a clear and accurate answer."], ['What is resistivity in current electricity?', 'Resistance in current electricity is a measure of how much a material or device opposes the flow of electric current. It is denoted by the symbol "R" and is measured in ohms (Ω). Resistance is influenced by factors such as the material of the conductor, its length, cross-sectional area, and temperature. Ohm\'s Law states that the current flowing through a conductor is directly proportional to the voltage applied across it and inversely proportional to the resistance of the conductor (I = V/R).']]
Adapter -> handle_user_message history: [['What is resistance in current electricity?', "I'm here to help with any physics-related questions you may have. Feel free to ask, and I'll do my best to provide you with a clear

/tmp/ipython-input-3743950133.py:33: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + "Z"


In [ ]:
# Run this right after you send the first question (and again after the second)
q = "What is resistance in current electricity?"   # change to the exact first/second question
# Call generate_answer with debug printing enabled (it prints MODEL MESSAGES)
ans, raw, src = generate_answer(q, history=[], memory_summary="", diagnostics={}, clarify=False)
print("=== ANSWER ===")
print("Source:", src)
print("Answer snippet:", (ans or "")[:400])
print("=== RAW REPLY ===")
print(raw)

=== MODEL MESSAGES ===
system : You are a concise, patient physics tutor for high-school and early-college students.
user : {user_message}
=== ANSWER ===
Source: nemo
Answer snippet: I'm here to help with any physics-related questions you may have. Feel free to ask, and I'll do my best to provide you with a clear and accurate answer.
=== RAW REPLY ===
{'role': 'assistant', 'content': "I'm here to help with any physics-related questions you may have. Feel free to ask, and I'll do my best to provide you with a clear and accurate answer."}


In [ ]:
# Add near where you created system_msg_template and chat_prompt_default
from langchain.prompts.chat import ChatPromptTemplate

# create a ChatPromptTemplate that contains only the system message
system_chat = ChatPromptTemplate.from_messages([system_msg_template])

In [ ]:
# OpenAI generator helper
import openai

def answer_query_with_openai(user_message: str, memory_summary: str = "", diagnostics: dict = None, clarify: bool = False):
    """
    Render system + user via prompt_mgr and call OpenAI ChatCompletion.
    Returns (assistant_text: str, raw_reply: dict)
    """
    system_text = prompt_mgr.render_system()
    user_text = prompt_mgr.render_user(user_message, clarify=clarify, memory_summary=memory_summary, diagnostics=diagnostics)

    messages = [
        {"role": "system", "content": system_text},
        {"role": "user", "content": user_text},
    ]

    try:
        resp = openai.ChatCompletion.create(
            model="gpt-3.5-turbo" ,
            messages=messages,
            temperature=0.0,
            max_tokens=2500,
        )
        choice = resp["choices"][0]["message"]
        assistant_text = choice.get("content", "")
        return assistant_text, resp
    except Exception as e:
        return f"Error generating response via OpenAI: {e}", {"error": str(e)}


In [ ]:
test_query = "resistance?"
ok, payload = vet_query(test_query)
print("vet ok:", ok)
if ok and payload.get("clarify"):
    clarif_payload = get_clarification_payload(test_query, memory_summary=payload.get("memory_summary",""), diagnostics=payload.get("diagnostics",{}))
    print("Clarification preview:", clarif_payload["preview"])
    # simulate user selecting a quick reply or editing the body; here we use the first quick reply
    user_confirmed_text = clarif_payload["quick_replies"][0]
    print("Simulating user confirmation:", user_confirmed_text)
    final = accept_clarification_and_generate(user_confirmed_text, original_user_message=test_query,
                                              diagnostics=payload.get("diagnostics",{}), memory_summary=payload.get("memory_summary",""))
    print("Final answer (snippet):", (final["answer"] or "")[:400])
else:
    print("No clarification required; full answer path or blocked.")


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 42.1MiB/s]


vet ok: True
No clarification required; full answer path or blocked.


In [ ]:
@traceable_function
def clear_memory():
    """Clear all conversation memory"""
    try:
        memory_collection.delete(where={"type": {"$ne": ""}})

        # Log memory clearance
        if langsmith_client:
            try:
                langsmith_client.create_feedback(
                    run_id=None,
                    key="memory_cleared",
                    score=1.0,
                    comment="All conversation memory cleared"
                )
            except Exception as e:
                print(f"LangSmith logging error: {e}")

        return "✅ Conversation memory cleared successfully!"
    except Exception as e:
        return f"❌ Error clearing memory: {str(e)}"

In [ ]:
@traceable_function
def export_memory():
    """Export conversation memory as JSON"""
    try:
        results = memory_collection.get()
        memories = []

        for i, doc in enumerate(results['documents']):
            memories.append({
                "content": doc,
                "metadata": results['metadatas'][i],
                "id": results['ids'][i]
            })

        filename = f"conversation_memory_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(filename, 'w') as f:
            json.dump(memories, f, indent=2)

        # Log export
        if langsmith_client:
            try:
                langsmith_client.create_feedback(
                    run_id=None,
                    key="memory_exported",
                    score=1.0,
                    comment=f"Exported {len(memories)} memories to {filename}"
                )
            except Exception as e:
                print(f"LangSmith logging error: {e}")

        return f"✅ Memory exported to {filename}"
    except Exception as e:
        return f"❌ Error exporting memory: {str(e)}"

In [ ]:
def get_guardrails_status():
    """Get status of safety guardrails"""
    if NEMO_GUARDRAILS_AVAILABLE and nemo_rails is not None:
        return "✅ NVIDIA NeMo Guardrails: ACTIVE"
    else:
        return "⚠️ NVIDIA NeMo Guardrails: NOT AVAILABLE"

def get_token_usage():
    """Get current token usage statistics"""
    summary = token_tracker.get_summary()
    return f"""
📊 Token Usage Summary:
• Total Tokens: {summary['total_tokens']:,}
• Prompt Tokens: {summary['prompt_tokens']:,}
• Completion Tokens: {summary['completion_tokens']:,}
• Estimated Cost: ${summary['estimated_cost']}
"""

In [ ]:
def reset_token_tracking():
    """Reset token tracking counters"""
    global token_tracker
    token_tracker = TokenUsageTracker()
    return "✅ Token tracking reset successfully!"

In [ ]:
def get_langsmith_status():
    """Get LangSmith tracing status"""
    if LANGCHAIN_AVAILABLE:
        return "✅ LangSmith Tracing: ACTIVE"
    else:
        return "⚠️ LangSmith Tracing: NOT AVAILABLE"

In [ ]:
def load_pdf_and_chat(pdf_file, message, history):
    # First store the PDF
    result_msg = store_pdf_in_db(pdf_file.name)

    # Then get response
    response = handle_user_message(message, history)

    return result_msg, response

In [ ]:
_pending_clarify = {}  # maps history_key -> { original_query, diagnostics, memory_summary }

def _history_key(history):
    try:
        return str(history[-6:])  # stable short key per session
    except Exception:
        return repr(history)

def _is_substantive_answer(text: str, min_len: int = 30) -> bool:
    if not text:
        return False
    t = text.strip().lower()
    # treat short greetings or canned tutor messages as non-substantive
    if len(t) < min_len:
        return False
    for prefix in ("hello", "hi", "hey", "i'm here to help", "feel free to ask"):
        if t.startswith(prefix):
            return False
    return True

def _last_entry_equals(history, user, assistant):
    if not history:
        return False
    last = history[-1]
    return len(last) >= 2 and last[0] == user and last[1] == assistant

def gradio_adapter(message, history):
    """
    Adapter for Gradio ChatInterface:
    - If a clarification is pending for this history, treat `message` as the confirmation text.
    - Otherwise call handle_user_message(message, history) and return a string.
    """
    print("gradio_adapter called:", message, "history_len:", len(history))
    global _pending_clarify
    key = _history_key(history)

    # 1) If pending clarification exists, treat this message as confirmation
    if key in _pending_clarify:
        pending = _pending_clarify.pop(key)
        final = accept_clarification_and_generate(
            clarification_text=message,
            original_user_message=pending.get("original_query"),
            diagnostics=pending.get("diagnostics", {}),
            memory_summary=pending.get("memory_summary", ""),
            history=history
        )
        answer = final.get("answer") or ""
        # Only append substantive answers to history
        if _is_substantive_answer(answer):
            history.append([message, answer])
        return answer or "No answer generated."

    # 2) Normal path: call handle_user_message and pass history
    result = handle_user_message(message, history=history)

    # Ensure result is a dict
    if not isinstance(result, dict):
        # fallback: return string
        reply = str(result)
        if _is_substantive_answer(reply):
            history.append([message, reply])
        return reply

    # 3) Clarify requested: store pending clarify and return UI payload
    if result.get("clarify"):
        _pending_clarify[key] = {
            "original_query": message,
            "diagnostics": result.get("diagnostics", {}),
            "memory_summary": result.get("memory_summary", "")
        }
        payload = result.get("clarification_payload", {})
        # Return the user-facing clarification body (or preview)
        return payload.get("body") or payload.get("preview") or "Could you clarify?"

    # 4) Normal answer path
    answer = result.get("answer") or _extract_text_from_reply(result.get("raw_reply"))
    raw = result.get("raw_reply")

    return answer or "I didn’t catch that clearly. Could you rephrase or be more specific?"

In [ ]:
history = []
gradio_adapter("What is resistance", history)
#gradio_adapter("What is resistivity in current electricity?", history)
#print("Follow-up (ambiguous):", gradio_adapter("give me their dimensions.", history))

gradio_adapter called: What is resistance history_len: 0
handle_user_message history len: 0 history: []
=== MODEL MESSAGES ===
system : You are a concise, patient physics tutor for high-school and early-college students.
user : You are a concise assistant that rewrites short or pronoun-based follow-up questions into explicit, self-contained questions using the conversation history. Do not add new facts. If the user message is already explicit, return it unchanged.

Conversation history:


User follow-up: What is resistance

Return a single-line rewritten question that is explicit (for example: 'dimensions of resistance and resistivity').
=== MODEL MESSAGES ===
system : You are a concise, patient physics tutor for high-school and early-college students.
user : What is resistance?


/tmp/ipython-input-3743950133.py:33: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat() + "Z"


"Resistance is a measure of how much a material or device opposes the flow of electric current. It is denoted by the symbol \\( R \\) and is measured in ohms (\\( \\Omega \\)). The higher the resistance, the more difficult it is for current to flow through the material or device. Resistance is influenced by factors such as the material's composition, length, cross-sectional area, and temperature."

In [ ]:
def launch_app():
    with gr.Blocks() as demo:
        gr.Markdown("# ⚛️ Physics Teacher Chat with PDF RAG + Web Search + Memory + Safety Guardrails + LangSmith Tracing")

        with gr.Row():
            with gr.Column():
                pdf_input = gr.File(label="Upload PDF", file_types=[".pdf"])
                load_btn = gr.Button("Load PDF into Knowledge Base")
                load_status = gr.Textbox(label="Load Status", interactive=False)

                gr.Markdown("### Safety & Memory Management")
                with gr.Row():
                    clear_mem_btn = gr.Button("🧹 Clear Memory")
                    export_mem_btn = gr.Button("💾 Export Memory")
                    safety_status_btn = gr.Button("🛡️ Safety Status")
                mem_status = gr.Textbox(label="Memory Status", interactive=False)

                gr.Markdown("### Token Usage Tracking")
                with gr.Row():
                    token_stats_btn = gr.Button("📊 Token Usage")
                    reset_tokens_btn = gr.Button("🔄 Reset Tracking")
                    tracing_status_btn = gr.Button("🔍 Tracing Status")
                token_status = gr.Textbox(label="Token Status", interactive=False)

                gr.Markdown("""
                ### How it works:
                1. **Upload a PDF** - Answers prioritize PDF content when relevant
                2. **Ask questions** - System uses PDF, web search, and past conversations
                3. **Long-term memory** - Remembers past conversations for context
                4. **Safety Guardrails** - NVIDIA NeMo Guardrails ensure safe, educational responses
                5. **Multiple sources** - Combines PDF 📚, Web 🔍, and Memory 💭
                6. **LangSmith Tracing** - Comprehensive tracing and token usage tracking
                7. **Token Monitoring** - Real-time token usage and cost tracking
                """)

            with gr.Column():
                chat_interface = gr.ChatInterface(
                    fn=gradio_adapter,
                    title="Chat with Physics Bot (with Memory, Safety & Tracing)",
                    description="Ask questions about physics. I'll remember our past conversations, use multiple knowledge sources, ensure safe responses, and track token usage!"
                )

        # Load PDF when button clicked
        load_btn.click(
            fn=lambda pdf: store_pdf_in_db(pdf.name) if pdf else "Please upload a PDF first",
            inputs=[pdf_input],
            outputs=[load_status]
        )

        # Memory management buttons
        clear_mem_btn.click(
            fn=clear_memory,
            outputs=[mem_status]
        )

        export_mem_btn.click(
            fn=export_memory,
            outputs=[mem_status]
        )

        safety_status_btn.click(
            fn=get_guardrails_status,
            outputs=[mem_status]
        )

        # Token tracking buttons
        token_stats_btn.click(
            fn=get_token_usage,
            outputs=[token_status]
        )

        reset_tokens_btn.click(
            fn=reset_token_tracking,
            outputs=[token_status]
        )

        tracing_status_btn.click(
            fn=get_langsmith_status,
            outputs=[token_status]
        )

    demo.launch()

if __name__ == "__main__":
    # Check for required API keys
    if not os.getenv("OPENAI_API_KEY"):
        print("❌ OPENAI_API_KEY not found in environment variables")
        exit(1)

    if not os.getenv("TAVILY_API_KEY") and TAVILY_AVAILABLE:
        print("❌ TAVILY_API_KEY not found in environment variables")
        print("Please get your free API key from: https://app.tavily.com/")
        print("Continuing without web search functionality...")

    # Check for LangSmith API key
    if not os.getenv("LANGCHAIN_API_KEY"):
        print("⚠️  LANGCHAIN_API_KEY not found. LangSmith tracing will be disabled.")
        print("💡 Get your free API key from: https://smith.langchain.com/")

    # Auto-load PDF if exists
    pdf_files = [f for f in os.listdir('.') if f.endswith('.pdf')]
    if pdf_files:
        print(f"Found PDF: {pdf_files[0]}")
        store_pdf_in_db(pdf_files[0])

    print("🧠 Long-term memory enabled!")
    print("📊 Token usage tracking enabled!")
    print(get_guardrails_status())
    print(get_langsmith_status())

    if LANGCHAIN_AVAILABLE:
        print("🔍 LangSmith tracing enabled! Visit https://smith.langchain.com/ to view traces.")

    launch_app()

🧠 Long-term memory enabled!
📊 Token usage tracking enabled!
✅ NVIDIA NeMo Guardrails: ACTIVE
✅ LangSmith Tracing: ACTIVE
🔍 LangSmith tracing enabled! Visit https://smith.langchain.com/ to view traces.


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f02e65bc4f975a2655.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
def _debug_handle_user_message(user_message):
    print("=== DEBUG handle_user_message start ===")
    print("user_message:", repr(user_message))

    # RAG check
    pdf_context = get_relevant_context(user_message, n_results=3)
    print("pdf_context (len):", len(pdf_context))
    print("is_pdf_context_relevant:", is_pdf_context_relevant(pdf_context, user_message))

    # vet_query
    ok, payload = vet_query(user_message)
    print("vet_query -> ok:", ok, "payload:", payload)

    # If vet_query says clarify False, call generator here and print result
    if ok and not payload.get("clarify", False):
        print("Proceeding to generate_answer() now...")
        memory_summary = payload.get("memory_summary", "")
        diagnostics = payload.get("diagnostics", {})
        ans_text, raw, source = generate_answer(user_message, memory_summary=memory_summary, diagnostics=diagnostics, clarify=False)
        print("generate_answer returned (snippet):", (ans_text or "")[:400])
        print("raw reply type:", type(raw))
        print("generator source:", source)
    else:
        print("Clarify required or vet_query blocked; payload clarify:", payload.get("clarify"))

    print("=== DEBUG handle_user_message end ===")
    return ok, payload

# Run it
_debug_handle_user_message("resistance?")


In [ ]:
# Direct generator test
ans, raw, src = generate_answer("Explain electrical resistance briefly and give one circuit example.", memory_summary="", diagnostics={}, clarify=False)
print("Answer snippet:", (ans or "")[:500])
print("Source:", src)
print("Raw reply type:", type(raw))

In [ ]:
# Temporary debug: print messages sent to the model
print("=== MODEL MESSAGES ===")
for m in messages:
    role = m.get("role")
    content = (m.get("content") or "")[:800]  # truncate
    print(f"{role}: {content!r}")
print("======================")

In [ ]:
print("Pending clarify map keys:", list(_pending_clarify.keys()))
print("PDF context for 'resistance?':", repr(get_relevant_context("resistance?", n_results=3)))
print("is_pdf_context_relevant:", is_pdf_context_relevant(get_relevant_context("resistance?", n_results=3), "resistance?"))
print("vet_query('resistance?'):", vet_query("resistance?"))